In [2]:
%load_ext autoreload
%autoreload 2

# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [3]:
import torch
import optuna
from optuna.visualization import plot_optimization_history, plot_contour, plot_parallel_coordinate, plot_slice
from pathlib import Path
from solver import Solver, TrialMetric
from griddy_tuna import hit_griddy, SearchMethod
from models.CRNN import CRNNModel
from data.data_loader import MirDataProcessor
from utils.model_utils import get_device

In [4]:
device = get_device()
print(f"Device is {device}")

Device is cuda


In [5]:
data_processer = MirDataProcessor(download=False, batch_size=128, dataset_name="guitarset", process_sequential=True, seq_length=16384)

data_processer.process_wav_data(150)

# Create data loeaders for train and test set
train_loader, test_loader, num_classes = data_processer.build_data_loaders(None, device)

print(f"Number of classes: {num_classes}")

Found 360 tracks in the dataset.
Processing dataset as sequential data
Processed track 00_BN1-129-Eb_comp and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN1-129-Eb_solo and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN1-147-Gb_comp and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN1-147-Gb_solo and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN2-131-B_comp and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN2-131-B_solo and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN2-166-Ab_comp and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN2-166-Ab_solo and appended data to combined CSV.
Processing dataset as sequential data
Processed track 00_BN3-119-G_comp and appended data to combined CSV

KeyboardInterrupt: 

In [9]:
from models.FourierS2S import FourierS2S

In [ ]:
# NOTE: Do not assume these values are anything but trash, they're just here for testing

SOLVER_PARAMS = {
    Solver : {
        "device": device,
        "batch_size": 256,
        "epochs": 50,
        "early_stop_epochs": 5, # early stop after n epochs without improvement, 0 to disable
        "warmup_epochs": 0, # 0 to disable
        "dtype": "float16",
        "train_dataloader": train_loader, # must be DataLoader object
        "valid_dataloader": test_loader, # must be DataLoader object
        "direction": "minimize" # must specify this, even if not used by solver
    }
}

MODEL_PARAMS = {
    FourierS2S: {
        "kernel_size": [512, 1024, 2048, 4096, 8192],
        "stride": [256, 512, 1024, 4096, 8192],
        "fourier_size": [128, 512, 2048, 4096, 8192],
        "enc_hidden": [64, 256, 512, 1024],
        "dec_hidden": [64, 256, 512, 1024],
        "output_size": [num_classes],
        "token_size": [16384],
        "device": [device]
    }
}

OPTIM_PARAMS = {

    torch.optim.Adam : {
        "lr": [0.05, 0.01, 0.1, .001, .0001], # this will auto-search as CATEGORICAL
    }
}

SCHED_PARAMS = {
    torch.optim.lr_scheduler.StepLR : {
        "step_size": [10, 20],
        "gamma" : [0.1, 0.01],
    }
}

CRITERION_PARAMS = {
    torch.nn.CrossEntropyLoss : {}
}

PARAM_SET = {
    "solver": SOLVER_PARAMS,
    "model" : MODEL_PARAMS,
    "optim" : OPTIM_PARAMS,
    "sched" : SCHED_PARAMS,
    "criterion" : CRITERION_PARAMS,
}

In [17]:
output_folder = Path("notebooks/my_study/griddy/")

solver_reference = hit_griddy("my_study", param_set=PARAM_SET, out_dir=output_folder, trial_metric=TrialMetric.LOSS, n_trials=20, n_jobs=1, prune=True, resume=False)

[I 2024-12-08 13:50:34,347] A new study created in RDB with name: my_study


"Hitting the griddy..." -Ellie
-----------------------------------
Epoch 1
-----------------------------------


Training:   0%|          | 0/3 [00:00<?, ?it/s]

Training Loss: 2.9493. Validation Loss: 3.0003.
Training Accuracy: 0.0169. Validation Accuracy: 0.1429.
-----------------------------------
Epoch 2
-----------------------------------


Training:   0%|          | 0/3 [00:00<?, ?it/s]

...

Training:   0%|          | 0/3 [00:00<?, ?it/s]

Training Loss: 3.0152. Validation Loss: 2.0214.
Training Accuracy: 0.0753. Validation Accuracy: 0.1429.
-----------------------------------
Epoch 20
-----------------------------------


Training:   0%|          | 0/3 [00:00<?, ?it/s]

[I 2024-12-08 13:58:39,999] Trial 19 finished with value: 67.99640774726868 and parameters: {'kernel_size': 512, 'stride': 256, 'fourier_size': 8192, 'enc_hidden': 64, 'dec_hidden': 256, 'lr': 0.1, 'step_size': 20, 'gamma': 0.1}. Best is trial 11 with value: 60.33701181411743.


Training Loss: 2.9978. Validation Loss: 2.0558.
Training Accuracy: 0.0445. Validation Accuracy: 0.1429.
DONE


In [18]:
for trial in solver_reference.trials:
    print(f"Trial {trial.number}:")
    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")
    print()

Trial 0:
  Value: 90.90808868408203
  Params: {'kernel_size': 1024, 'stride': 4096, 'fourier_size': 512, 'enc_hidden': 4, 'dec_hidden': 512, 'lr': 0.0001, 'step_size': 10, 'gamma': 0.01}

Trial 1:
  Value: 109.52500343322754
  Params: {'kernel_size': 8192, 'stride': 1024, 'fourier_size': 128, 'enc_hidden': 4, 'dec_hidden': 16, 'lr': 0.0001, 'step_size': 10, 'gamma': 0.1}

Trial 2:
  Value: 96.9582188129425
  Params: {'kernel_size': 8192, 'stride': 256, 'fourier_size': 4096, 'enc_hidden': 256, 'dec_hidden': 4, 'lr': 0.01, 'step_size': 10, 'gamma': 0.01}

Trial 3:
  Value: 78.18519234657288
  Params: {'kernel_size': 4096, 'stride': 8192, 'fourier_size': 4096, 'enc_hidden': 64, 'dec_hidden': 256, 'lr': 0.1, 'step_size': 10, 'gamma': 0.1}

Trial 4:
  Value: 88.49106073379517
  Params: {'kernel_size': 4096, 'stride': 8192, 'fourier_size': 128, 'enc_hidden': 16, 'dec_hidden': 4, 'lr': 0.01, 'step_size': 20, 'gamma': 0.1}

Trial 5:
  Value: 65.51632761955261
  Params: {'kernel_size': 1024, 's

In [17]:
import optuna

full_path = os.path.join(output_folder, "my_study.db")
loaded_study = optuna.load_study(study_name="my_study", storage=f"sqlite:///{full_path}")

Trial 0:
  Value: None
  Params: {'kernel_size': 4096, 'stride': 1024, 'fourier_size': 2048, 'enc_hidden': 256, 'dec_hidden': 16, 'lr': 0.01, 'step_size': 10, 'gamma': 0.1}

Trial 1:
  Value: None
  Params: {'kernel_size': 8192, 'stride': 4096, 'fourier_size': 128, 'enc_hidden': 256, 'dec_hidden': 512, 'lr': 0.0001, 'step_size': 20, 'gamma': 0.1}

Trial 2:
  Value: None
  Params: {'kernel_size': 8192, 'stride': 4096, 'fourier_size': 8192, 'enc_hidden': 256, 'dec_hidden': 256, 'lr': 0.0001, 'step_size': 20, 'gamma': 0.1}

Trial 3:
  Value: None
  Params: {'kernel_size': 8192, 'stride': 1024, 'fourier_size': 8192, 'enc_hidden': 64, 'dec_hidden': 512, 'lr': 0.05, 'step_size': 20, 'gamma': 0.1}

Trial 4:
  Value: None
  Params: {'kernel_size': 1024, 'stride': 8192, 'fourier_size': 4096, 'enc_hidden': 256, 'dec_hidden': 256, 'lr': 0.1, 'step_size': 20, 'gamma': 0.01}

Trial 5:
  Value: None
  Params: {'kernel_size': 512, 'stride': 1024, 'fourier_size': 2048, 'enc_hidden': 512, 'dec_hidden':